In [85]:
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import pandas as pd

tag2prefix = {'ABDOMINAL':'AB', 'ADVANCED-CAD':'AD', 'ALCOHOL-ABUSE':'AL', 'ASP-FOR-MI':'AS', 'CREATININE':'CR', \
        'DIETSUPP-2MOS':'DI', 'DRUG-ABUSE':'DR', 'ENGLISH':'EN', 'HBA1C':'HB', \
        'MAJOR-DIABETES':'MA', 'MAKES-DECISIONS':'MK', 'MI-6MOS':'MI', 'KETO-1YR':'KE' }
tag2id = {'ABDOMINAL':0, 'ADVANCED-CAD':0, 'ALCOHOL-ABUSE':0, 'ASP-FOR-MI':0, 'CREATININE':0, \
        'DIETSUPP-2MOS':0, 'DRUG-ABUSE':0, 'ENGLISH':0, 'HBA1C':0, \
        'MAJOR-DIABETES':0, 'MAKES-DECISIONS':0, 'MI-6MOS':0, 'KETO-1YR':0 }

def checkKETOandHBA1C(train_df):
    result_file = open("result.txt", 'w+') 
    df_record = train_df['record_text'].copy()
    idprefix = tag2prefix["HBA1C"]
    idnum = tag2id["HBA1C"]
    def preprocessing(text):
        removelist = ['\n','\t']
        tokens = word_tokenize(text.lower())
        processed = []
        pattern = re.compile(r"\d+-\d+-\d+")
        pattern2 = re.compile(r"\d+/\d+/\d+")
        pattern3 = re.compile(r"was")
        pattern4 = re.compile(r"\d+/\d+")
        for each in tokens:
            if each not in removelist:            
                match = pattern.search(each)
                match2 = pattern2.search(each)
                match3 = pattern3.search(each)
                match4 = pattern4.search(each)
                if not match and not match2 and not match3 and not match4:
                    processed.append(each)
        res = ' '.join(processed)
        res = re.sub('[^a-z0-9.%]',' ',res)
        res = re.sub(r'\s+', '', res)
        return res
    for i, each in enumerate(df_record):
        df_record[i] = preprocessing(each)
    #def checkKETO(df_record):

    def checkHBA1C(df_record):
        HBA1C_MET = []
        flag = 0
        flag_group = []
        for each in df_record:
            for num in range(0,20):
                pattern = re.compile(r"(?<=a1c[a-z0-9\s]{"+str(num)+"})\d+\.\d{0,2}")
                pattern2 = re.compile(r"(?<=a1c[a-z0-9\s]{"+str(num)+"})\d\.?\d{0,2}(?=%)")
                match = pattern.findall(each)
                match2 = pattern2.findall(each)
                if match:
                    #print(flag)
                    #print(match)
                    for each in match:
                        float_each = float(each)
                        if float_each >= 6.5 and float_each <= 9.5:
                            flag_group.append(flag)
                            HBA1C_MET.append(match)
                            id = idprefix + str(tag2id["HBA1C"])                    
                            print(train_df.loc[flag,'record_file']+", HBA1C, " +id+", met", file=result_file)
                            tag2id["HBA1C"] += 1
                if match2:
                    for each in match2:
                        float_each = float(each)
                        if float_each >= 6.5 and float_each <= 9.5:
                            flag_group.append(flag)
                            HBA1C_MET.append(match2)              
                            id = idprefix + str(tag2id["HBA1C"])
                            print(train_df.loc[flag,'record_file']+", HBA1C, " + id+", met", file=result_file)
                            tag2id["HBA1C"] += 1
            if flag not in flag_group:
                print(train_df.loc[flag,'record_file']+", HBA1C, HB0, not met", file=result_file)
            flag =flag + 1
            tag2id["HBA1C"] = 0
        return flag_group    
    #KETO = checkKETO(df_record)
    HBA1C = checkHBA1C(df_record)
    return HBA1C

In [86]:
train_df = pd.read_csv("all-train.csv")
test_df = pd.read_csv("all-test.csv")
a = checkKETOandHBA1C(test_df)
print(a)

[5, 5, 10, 20, 22, 22, 24, 27, 28, 30, 33, 33, 33, 34, 35, 37, 37, 40, 42, 45, 48, 48, 48, 48, 48, 48, 48, 48, 50, 60, 62, 67, 67, 69, 70, 71, 75, 75, 75, 76, 76, 77, 77, 77, 79, 79, 79, 81, 83]
